Analyzing borrowers’ risk of defaulting
======

**Goal**: to test four hypotheses about the bank's clients to formulate recommendations for creation of a credit scoring model.

The project investigated data on the solvency of clients, taking into account the timely repayment of the loan. During the analysis, erroneous artifacts were eliminated, missing values were restored, and data was transformed to test hypotheses about a connection between a certain customer characteristic and their possible delay in loan repayment.

Four hypotheses were tested:
- Is there a connection between having kids and repaying a loan on time?
- Is there a connection between marital status and repaying a loan on time? 
- Is there a connection between income level and repaying a loan on time? 
- How do different loan purposes affect timely loan repayment?

At the end of the project, general conclusions and recommendations are given.

**Description for student:**
Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building the credit score of a potential customer. The credit score is used to evaluate the ability of a potential borrower to repay their loan.

[In this notebook you're provided with hints and brief instructions and thinking prompts. Don't ignore them as they are designed to equip you with the structure for the project and will help you analyze what you're doing on a deeper level. Before submitting your project, make sure you remove all hints and descriptions provided to you. Instead, make this report look as if you're sending it to your teammates to demonstrate your findings - they shouldn't know you had some external help from us! To help you out, we've placed the hints you should remove in square brackets.]

[Before you dive into analyzing your data, explain the purposes of the project and hypotheses you're going to test.]

In [48]:
# Loading required libraries
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
import pprint
import matplotlib.pyplot as plt

In [6]:
# Loading the data
data = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Task 1. Data exploration

**Description of the data**

children: the number of children in the family

days_employed: how long the customer has been working

dob_years: the customer’s age

education: the customer’s education level

education_id: identifier for the customer’s education

family_status: the customer’s marital status

family_status_id: identifier for the customer’s marital status

gender: the customer’s gender

income_type: the customer’s income type

debt: whether the customer has ever defaulted on a loan

total_income: monthly income

purpose: reason for taking out a loan

In [7]:
# Data table contains 21525 rows and 12 columns
data.shape

(21525, 12)

In [8]:
# Data table contains 21525 rows and 12 columns
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


During the initial exploration of the dataset, we see both positive and negative values in days_employed, as well as missing values of days_employed and total_income. You can also see that the values of the type of education are written in a different case.

In [9]:
# Crecking the information on the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Only two columns have missing data. Data may be missing symmetrically in two columns per row.

In [10]:
# Let's look at the table with missing values in the days_employed column
data[data['days_employed'].isnull()].head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing


At first glance, missing values in the *days_employed* and *total_income* columns are indeed symmetric. 
To be sure of this assumption, you need to filter the table by missing values in rows for both columns. 
And immediately count the number of rows in the table filtered by missing values. 
If the number of rows coincides with the number of missing values, then the assumption about the symmetry of missing values in two  columns is confirmed.


In [54]:
# We apply two conditions for filtering data and look at the number of rows in the filtered table.
data[(data['days_employed'].isnull()) & (data['total_income'].isnull())].shape

(2174, 12)

**Intermediate conclusion**

The number of rows in the filtered table matched the number of missing values. 
Thus, missing values in two columns are symmetrical.

**It is necessary to notify colleagues** who are involved in the creation of reports about missing values in the data and about their symmetric absence in the columns.

10% of clients missed information is on days_employed and total_income. 
Removing such a fraction of the data can distort the study results. 
Thus, we will select the method for filling the missing values.
First, you should consider the nature of the missing values. 
Symmetrical missing values may reflect specific client characteristic. 
One of the main indicators when deciding whether to issue a loan is the type of income and the client's age. 
Therefore, we will start checking the dependence of missing values on the value of other indicators with the columns income_type (type of employment) and dob_years (age of the client in years).


In [12]:
# Let's investigate clients who do not have data on both work experience and monthly income
data_nan = data[data['days_employed'].isnull()]

In [13]:
# Checking distribution by income_type
data_nan['income_type'].value_counts(normalize = True)

employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64

Information on days_employed and total_income is mostly absent for employed employees, entrepreneurs and retirees. 
Let's display the distribution of the original dataset to check for the randomness of missing values.

**Possible reasons for missing values in data** 

The lack of days_employed and total_income among retirees may be due to the fact that clients at their age have income only as a pension and have already written an application for a loan during their retirement, i.e. they did not have any total_income/days_employed at the time of the loan application. 
It is possible that the client's total_income indicates private income rather than government pension.

**It is necessary to request** detailed metadata of the dataset from colleagues who create reports.


In [14]:
# Checking the distribution of income_type based on the number of cases 
data['income_type'].value_counts(normalize = True)

employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: income_type, dtype: float64

The distribution of income_type in the original dataset is very similar to the distribution of the same parameter in a table sorted by missing values.

**Intermediate conclusion**

This situation may indicate that values are missed randomly. The data in the table with missing values is evenly distributed if comparing to the original dataset. To test that the missing values depend on other indicators, let's look at the data on clients who, in theory, can also receive government benefits: clients on maternity leave and students. Let's also compare the data for entrepreneurs. There are now may clients from this category - only two. One of them is missing data.

In [16]:
data[(data['income_type'] == 'student') | (data['income_type'] == 'paternity / maternity leave') |\
     (data['income_type'] == 'unemployed') | (data['income_type'] == 'entrepreneur')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,secondary education,1,married,0,M,unemployed,1,9593.119,buying property for renting out
5936,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate
9410,0,-578.751554,22,bachelor's degree,0,unmarried,4,M,student,0,15712.260,construction of own property
14798,0,395302.838654,45,Bachelor's Degree,0,civil partnership,1,F,unemployed,0,32435.602,housing renovation
18697,0,-520.848083,27,bachelor's degree,0,civil partnership,1,F,entrepreneur,0,79866.103,having a wedding
20845,2,-3296.759962,39,SECONDARY EDUCATION,1,married,0,F,paternity / maternity leave,1,8612.661,car


**Intermediate conclusion**

The assumption that missing values may not be accidental is not confirmed.
The data on entrepreneur is missing.
Let's examine data with missing values regarding the type of income and the age of clients. 
Perhaps there is a relationship between these customer characteristics. 
To do this, let's create a pivot table.


In [17]:
# Creating a pivot table
data_nan_pivot = data_nan.pivot_table(index='dob_years', columns='income_type', values='debt', aggfunc='count', margins=True)

In [18]:
# Filtering the pivot table by the total number of customers regarding their age, sorting by total number
data_nan_pivot.sort_values(by='All', ascending=False)

income_type,business,civil servant,employee,entrepreneur,retiree,All
dob_years,,,,,,
All,508.0,147.0,1105.0,1.0,413.0,2174
34,22.0,4.0,43.0,NaN,NaN,69
40,26.0,9.0,30.0,NaN,1.0,66
42,20.0,1.0,40.0,NaN,4.0,65
31,28.0,7.0,29.0,NaN,1.0,65
35,25.0,2.0,37.0,NaN,NaN,64
36,16.0,9.0,36.0,NaN,2.0,63
47,16.0,5.0,38.0,NaN,NaN,59
41,14.0,2.0,42.0,NaN,1.0,59


In [20]:
# Checking patterns among clients regarding age and type of employment, sorting by type of employment - employee
data_nan_pivot.sort_values(by='employee', ascending=False)

income_type,business,civil servant,employee,entrepreneur,retiree,All
dob_years,,,,,,
All,508.0,147.0,1105.0,1.0,413.0,2174
30,10.0,4.0,44.0,NaN,NaN,58
34,22.0,4.0,43.0,NaN,NaN,69
41,14.0,2.0,42.0,NaN,1.0,59
42,20.0,1.0,40.0,NaN,4.0,65
47,16.0,5.0,38.0,NaN,NaN,59
35,25.0,2.0,37.0,NaN,NaN,64
49,9.0,3.0,36.0,NaN,2.0,50
36,16.0,9.0,36.0,NaN,2.0,63


**Intermediate conclusion**

There is no seen relationship between the client's age and type of employment. 
The summary table shows that the majority of clients who do not have data on income and days_employed are over the age of 30 and work as employees and entrepreneurs.
So the data is missing randomly. 

For accurate restoration of missing values, we will focus on the most common values, depending on the age, education and type of income of the client.

To simplify calculations, we will split clients by age categories (a group for every 20 years):

- Up to 25 years old
- 26-45 years old
- 46-65 years old
- 66 years old and older

**Note**. 

Since the recovery of data on monthly income will be based on three columns, in which there are duplicates, incorrect artifacts, first, we will deal with data transformation and then restore missing values.
Using pivot tables, we found that the column with customer ages contains zero values. 
It is also necessary to check the data for duplicates and bring the column with data on education into one form.


## Data transformation

1) Removing duplicates in the dataset and converting customer education information.


In [21]:
# Let's check all duplicates in 'education' due to different case 
data['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [23]:
# Counting duplicates in the dataset
data.duplicated().sum()

54

In [24]:
# Counting duplicates in the dataset
data['education'] = data['education'].str.lower()

In [25]:
# Checking the variety of values of clients' information about education
data['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

In [26]:
# Counting duplicates in the dataset
data.duplicated().sum()

71

After the transformation of education column, the number of duplicates increased. 
Therefore, it will be more optimal to transform the remaining columns, where there are problematic artifacts. 
And after the transformation, we will remove duplicates from the dataset.
Let's check the data in children column.


In [27]:
# Let's see the distribution of values in the column 
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

The column contains abnormal artifacts: -1 and 20. 
Probably, these errors appeared due to an error of the manager who entered the information about the client. 
These errors in the dataset should also be reported to colleagues. 
Erroneous data is less than 1% of the sample. 
You can delete them.


<div class="alert alert-block alert-info">
<b>Senior reviewer comment</b> <a class="tocSkip"></a>
    
Also we can say -1 = 1 and 20 = 2. </div>

In [28]:
#drop values
data.drop(data[data['children'] == 20].index, inplace=True)
data.drop(data[data['children'] == -1].index, inplace=True)

In [29]:
# Deleting these values
data['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Transforming data in days_employed

In [30]:
print('Rows with negative value in days_employed is ', data[data['days_employed'] < 0].shape[0], 'rows')
print(f"The percentage of rows with negative values in days_employed of the total dataset is N  {data[data['days_employed'] < 0].shape[0]/data.shape[0]:.2%} rows")

Rows with negative value in days_employed is  15809 rows
The percentage of rows with negative values in days_employed of the total dataset is N  73.87% rows


Negative data in days_employed could have appeared due to technical errors while loading the data

In [31]:
# Let's transform the data: take the modulus of the number
data['days_employed'] = data['days_employed'].abs()

In [32]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [33]:
# Counting duplicates in the dataset
data.duplicated().sum()

71

Checking clients' age

In [34]:
# We've noticed before that some clients have their age equal to 0
data['dob_years'].value_counts().sort_index().head() 

0     100
19     14
20     51
21    110
22    183
Name: dob_years, dtype: int64

Less than 1% of clients have age equal to zero. 
Let's point out to our colleagues about this error in the data. 
Most likely, this error occurred while entering customer information into the database.


In [35]:
# this value somehow appeared in our data
data.drop(data[data['dob_years'] == 0].index, inplace=True)

In [36]:
#checking results
data[data['dob_years'] == 0]['dob_years'].count()

0

Checking maritial status

In [37]:
# Printing the unique values in the column
data['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

'married', 'civil marriage', 'widower/widow','divorced', 'Single' 

Although the status "unmarried" is unique, it is not written same as other values.
We need to transform everything to lower case.


In [38]:
# Lower-case values from the family_status
data['family_status'] = data['family_status'].str.lower()

In [39]:
# Counting duplicates in the dataset
data.duplicated().sum()

71

Let's check gender

In [40]:
data['gender'].value_counts()

F      14083
M       7218
XNA        1
Name: gender, dtype: int64

We need to delete the row with erroneous value. 

In [41]:
# Removing the row
data.drop(data[data['gender'] == 'XNA'].index, inplace=True)

In [42]:
# Checking the result
data['gender'].value_counts()

F    14083
M     7218
Name: gender, dtype: int64

Checking income type

In [43]:
data['income_type'].value_counts()

employee                       10996
business                        5033
retiree                         3819
civil servant                   1447
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

'worker' 'companion' 'pensioner''civil servant' 'entrepreneur'

The data does not contain erroneous values. 
You can proceed to removing duplicates from the dataset.


In [44]:
# checking retirees 
data[data['income_type'] == 'retiree']['dob_years'].value_counts().sort_index().head(10)

22    1
24    1
26    2
27    3
31    1
32    3
33    2
34    3
35    1
36    5
Name: dob_years, dtype: int64

The distribution of clients by age with the type of income "retired" shows that there are extreme values of age: 22 years old or 33 years old. However, some of these clients may be military personnel who retired early, or disabled people who receive disability benefits. 
Let's leave this data for further calculations.


In [45]:
# Counting duplicates in the dataset
data.duplicated().sum()

71

In [46]:
data = data.drop_duplicates().reset_index(drop=True)

In [47]:
# Last check - all duplicated are gone
data.duplicated().sum()

0

In [48]:
data.shape

(21230, 12)

Initially there were 21525 rows in the dataset. 
There are 21230 rows left after data transformation. 
1.4% of customer information was removed.


# Working with missing values



The dataset contains dictionaries - digital designation of data for faster work with the dataset.

In [49]:
# Dictionary for education type
education_dict = data[['education_id', 'education']]
education_dict= education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,bachelor's degree
1,1,secondary education
2,2,some college
3,3,primary education
4,4,graduate degree


In [50]:
# Dictionary for maritial status
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

In [51]:
family_status_dict

,family_status_id,family_status
0,0,married
1,1,civil partnership
2,2,widow / widower
3,3,divorced
4,4,unmarried


### Restoring missing values in income

Let's create an age category for a more convenient calculation for the missing values in the **monthly income column**.
To simplify calculations, we will divide clients into age categories:

- Up to 25 years old
- 26-45 years old
- 46-65 years old
- 66 years old and older


In [53]:
def age_group(age):
    """
        It returns the age category given the age value, by the following rules:
        - Up to 25 years old if age < 25
        - 26-45 years old if 26 < age <45
        - 46-65 years old if 46 < age <65
        - 66 years old and older if age > 66
    """

    try:
        if age <= 25:
            return 'Up to 25 years old'
        if 26 <= age <= 45:
            return '26-45 years old'
        if 46 <= age <= 65:
            return '46-65 years old'
        else:
            return '66 years old and older'
    except:
        return 0
    

In [54]:
# Testing the function
age_group(75)

'66 years old and older'

In [55]:
# Creating a new column based on the function
data['age_group'] = data['dob_years'].apply(age_group)

In [56]:
# Checking how it worked
data['age_group'].value_counts()

26-45 years old           10900
46-65 years old            8404
Up to 25 years old         1226
66 years old and older      700
Name: age_group, dtype: int64

Usually, the income depends on three characteristics: age, education, and type of employment.
Let's first check the distribution of age and type of income using pivot tables. 
It is necessary to find out which values (mean or median) are better for replacing missing values.


Let's create a table that has only the data without missing values to calculate the missing values in two columns. 
Restoring the values in the days_employed and total_income will be based on several characteristics of the client.


In [57]:
data_without_nan = data[data['days_employed'].isnull() != True]

In [58]:
data_without_nan.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,26-45 years old
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,26-45 years old
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26-45 years old
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26-45 years old
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,26-45 years old
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,26-45 years old
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,46-65 years old
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,26-45 years old
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,26-45 years old


In [59]:
# A pivot table by income type and age group
data_without_nan.pivot_table(index='income_type', columns='age_group', values='total_income', aggfunc='mean')

age_group,26-45 years old,46-65 years old,66 years old and older,Up to 25 years old
income_type,,,,
business,32930.411643,33051.369974,33470.062000,25752.351209
civil servant,27442.064958,27499.138709,30992.299000,24510.974242
employee,26128.693783,26015.185750,26185.024870,22265.980563
entrepreneur,79866.103000,NaN,NaN,NaN
paternity / maternity leave,8612.661000,NaN,NaN,NaN
retiree,24408.993847,22343.926661,19663.470405,14298.976000
student,NaN,NaN,NaN,15712.260000
unemployed,21014.360500,NaN,NaN,NaN


In [60]:
# A pivot table by income type and age group
data_without_nan.pivot_table(index='income_type', columns='age_group', values='total_income', aggfunc='median')

age_group,26-45 years old,46-65 years old,66 years old and older,Up to 25 years old
income_type,,,,
business,28094.4310,28406.377,29314.4045,22814.5995
civil servant,24368.0150,23847.285,26089.6870,22758.5535
employee,23066.1730,22781.846,24643.1985,20634.6650
entrepreneur,79866.1030,NaN,NaN,NaN
paternity / maternity leave,8612.6610,NaN,NaN,NaN
retiree,20028.7250,19420.007,17074.5790,14298.9760
student,NaN,NaN,NaN,15712.2600
unemployed,21014.3605,NaN,NaN,NaN


We will take the median, since the mean is highly dependent on outliers.

This way the values of clients' monthly income will be less dependent on the extreme values in the group.
Let's also check if the distribution of income depends on education and type of income and compare it to the pivot table with distributions on age and type of employment. 

We need this table to compare how income can vary depending on different client's characteristics.

In [61]:
data_without_nan.pivot_table(index='income_type', columns='education', values='total_income', aggfunc='median')

education,bachelor's degree,graduate degree,primary education,secondary education,some college
income_type,,,,,
business,32285.664,NaN,21887.825,25441.230,28778.744
civil servant,27564.459,17822.757,23734.287,21864.475,25694.775
employee,26587.423,31771.321,20159.186,21841.813,24209.430
entrepreneur,79866.103,NaN,NaN,NaN,NaN
paternity / maternity leave,NaN,NaN,NaN,8612.661,NaN
retiree,23030.247,28334.215,16415.785,18372.071,19221.903
student,15712.260,NaN,NaN,NaN,NaN
unemployed,32435.602,NaN,NaN,9593.119,NaN


In [65]:
# Pivot table based on three characteristics: age, education, and type of income  
data_without_nan.pivot_table(index=['age_group','income_type' ], columns='education', values='total_income', \
                             aggfunc='median')

education                                           bachelor's degree  \
age_group              income_type                                      
26-45 years old        business                            32348.9660   
                       civil servant                       27034.6050   
                       employee                            26766.0260   
                       entrepreneur                        79866.1030   
                       paternity / maternity leave                NaN   
                       retiree                             30117.8830   
                       unemployed                          32435.6020   
46-65 years old        business                            34807.6890   
                       civil servant                       29517.2390   
                       employee                            27847.3160   
                       retiree                             23933.6670   
66 years old and older business                            36808.9680   
                       civil servant                       31984.6160   
                       employee                            25137.4170   
                       retiree                             21049.8135   
Up to 25 years old     business                            24721.9040   
                       civil servant                       23839.4605   
                       employee                            21560.6225   
                       retiree                                    NaN   
                       student                             15712.2600   

education                                           graduate degree  \
age_group              income_type                                    
26-45 years old        business                                 NaN   
                       civil servant                     17822.7570   
                       employee                          25161.5835   
                       entrepreneur                             NaN   
                       paternity / maternity leave              NaN   
                       retiree                                  NaN   
                       unemployed                               NaN   
46-65 years old        business                                 NaN   
                       civil servant                            NaN   
                       employee                          42945.7940   
                       retiree                           40868.0310   
66 years old and older business                                 NaN   
                       civil servant                            NaN   
                       employee                                 NaN   
                       retiree                           15800.3990   
Up to 25 years old     business                                 NaN   
                       civil servant                            NaN   
                       employee                                 NaN   
                       retiree                                  NaN   
                       student                                  NaN   

education                                           primary education  \
age_group              income_type                                      
26-45 years old        business                            21664.8730   
                       civil servant                       30545.9490   
                       employee                            19810.2530   
                       entrepreneur                               NaN   
                       paternity / maternity leave                NaN   
                       retiree                             17973.2670   
                       unemployed                                 NaN   
46-65 years old        business                            26144.4830   
                       civil servant                       16922.6250   
                       empl

The client's median income is best described by these three characteristics.

In [66]:
# Pivot table for calculating the median monthly income  
pivot_table_for_total_income = data_without_nan.pivot_table(index=['age_group','income_type' ], columns='education', values='total_income', \
                             aggfunc='median')


def fill_nan_total_income(age, income_type, education):
    """
    A function used to calculate the median monthly income of a client based on their age group, type of income and education.
    """
    try:
        return pivot_table_for_total_income[education][age][income_type]
    except:
        return 'error'
        


In [69]:
# Testing the function
pivot_table_for_total_income["bachelor's degree"]['Up to 25 years old']['civil servant']

23839.4605

In [70]:
# fill in the median value for each row in a new column
data['median_total_income'] = data.apply(lambda row: fill_nan_total_income(row['age_group'], row['income_type'], \
                                                           row['education']), axis=1)

In [71]:
# Test how the function works.Are there any monthly income values that have not been found?
data[data['median_total_income'] == 'error']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_total_income
5880,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,46-65 years old,error


We couldn't find the median income for entrepreneur. We only have two clients of this type in the dataset. 
Manually setting the the income of the second entrepreneur same as the one of the first. 
But first, let's replace all missing values in the `total_income` column with the corresponding values from the `median_total_income` column.


In [73]:
### Restoring missing values in total_income
data['total_income'] = data['total_income'].fillna(data['median_total_income'])

In [74]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_total_income
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,26-45 years old,26766.026
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,26-45 years old,22047.114
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26-45 years old,22047.114
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26-45 years old,22047.114
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old,18749.915
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.17,purchase of the house,26-45 years old,32348.966
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,26-45 years old,32348.966
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,46-65 years old,21741.8725
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,26-45 years old,26766.026
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.15,purchase of the house for my family,26-45 years old,22047.114


Monthly income values became non-floating, because there appeared a string value in the column ("error")

In [75]:
# Manually setting the the income of the second entrepreneur 
data.loc[data['total_income'] == 'error', ['total_income']] = 499163

In [77]:
data[data['income_type'] == 'entrepreneur']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_total_income
5880,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,499163,buy residential real estate,46-65 years old,error
18450,0,520.848083,27,bachelor's degree,0,civil partnership,1,F,entrepreneur,0,79866.103,having a wedding,26-45 years old,79866.103


Changing the data type of the column to integer. 
We use the .astype() method, because we need a specific data type.
If you use the .to_numeric() method, then all numbers will become floats again.


In [78]:
# Changing data type
data['total_income'] = data['total_income'].astype('int')

In [79]:
#test
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   children             21230 non-null  int64  
 1   days_employed        19149 non-null  float64
 2   dob_years            21230 non-null  int64  
 3   education            21230 non-null  object 
 4   education_id         21230 non-null  int64  
 5   family_status        21230 non-null  object 
 6   family_status_id     21230 non-null  int64  
 7   gender               21230 non-null  object 
 8   income_type          21230 non-null  object 
 9   debt                 21230 non-null  int64  
 10  total_income         21230 non-null  int32  
 11  purpose              21230 non-null  object 
 12  age_group            21230 non-null  object 
 13  median_total_income  21230 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(7)
memory usage: 2.2+ MB


<div class="alert alert-block alert-info">
<b>Senior Reviewer's comment</b> <a class="tocSkip"></a>
    
Only one column can be used to fill in the missing values.</div>

###  Restoring missing values in days_employed

Let's check which combination of client characteristics can generate the most typical data for the recovery of the missing data.

In [80]:
# Distribution of working experience values by type of income and age group. Mean.
data_without_nan.pivot_table(index='income_type', columns='age_group', values='days_employed', aggfunc='median')

age_group,26-45 years old,46-65 years old,66 years old and older,Up to 25 years old
income_type,,,,
business,1484.965033,2086.501856,2318.709538,748.818654
civil servant,2521.457009,3601.346755,4137.331615,1132.739641
employee,1513.345802,2184.737485,2830.361431,798.699314
entrepreneur,520.848083,NaN,NaN,NaN
paternity / maternity leave,3296.759962,NaN,NaN,NaN
retiree,364348.197352,365062.716683,366157.236636,334764.259831
student,NaN,NaN,NaN,578.751554
unemployed,366413.652744,NaN,NaN,NaN


In [81]:
# Distribution of working experience values by type of income and age group. Median.
data_without_nan.pivot_table(index='income_type', columns='age_group', values='days_employed', aggfunc='mean')

age_group,26-45 years old,46-65 years old,66 years old and older,Up to 25 years old
income_type,,,,
business,1884.455090,2898.938303,3725.387000,861.980178
civil servant,2940.240153,4583.807565,4145.742201,1174.478880
employee,2067.795829,3148.116338,4092.413329,929.403103
entrepreneur,520.848083,NaN,NaN,NaN
paternity / maternity leave,3296.759962,NaN,NaN,NaN
retiree,364295.980201,364941.071441,365676.375980,334764.259831
student,NaN,NaN,NaN,578.751554
unemployed,366413.652744,NaN,NaN,NaN


In [82]:
# Distribution of working experience values by type of income using median values.
data.groupby('income_type')['days_employed'].median()

income_type
business                         1555.993659
civil servant                    2672.903939
employee                         1573.791064
entrepreneur                      520.848083
paternity / maternity leave      3296.759962
retiree                        365269.100414
student                           578.751554
unemployed                     366413.652744
Name: days_employed, dtype: float64

The median values of the type of income but in the context of the type of income of the client sufficiently describe the typical values. 
Restoring values by seniority work experience will only be relative to one characteristic of the client.


In [85]:
# Distribution of data into groups by type of employment to restore the work experience value 
groupby_income_type_for_days_employed = data.groupby('income_type')['days_employed'].median()

def fill_nan_days_employed(income_type):
    """
    Functions to restore the work experience values.
    It is based on the median value and the distribution of clients' income type.

    """
    try:
        return groupby_income_type_for_days_employed[income_type]
    except:
        return 'error'

In [86]:
# Testing the function
fill_nan_days_employed('student')

578.7515535382181

In [87]:
# fill in the median value for each row in a new column
data['median_days_employed'] = data['income_type'].apply(fill_nan_days_employed)

In [88]:
# Testing how the function works
data['median_days_employed'].value_counts()

1573.791064      10961
1555.993659       5026
365269.100414     3792
2672.903939       1445
366413.652744        2
520.848083           2
578.751554           1
3296.759962          1
Name: median_days_employed, dtype: int64

In [89]:
# Filling in the missing values in the work experience of clients in the main dataset
data['days_employed'] = data['days_employed'].fillna(data['median_days_employed'])

In [90]:
data.isnull().sum()

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
age_group               0
median_total_income     0
median_days_employed    0
dtype: int64

In [91]:
# Transforming the work experience to integers
data['days_employed'] = data['days_employed'].astype('int')

In [92]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21230 non-null  int64  
 1   days_employed         21230 non-null  int32  
 2   dob_years             21230 non-null  int64  
 3   education             21230 non-null  object 
 4   education_id          21230 non-null  int64  
 5   family_status         21230 non-null  object 
 6   family_status_id      21230 non-null  int64  
 7   gender                21230 non-null  object 
 8   income_type           21230 non-null  object 
 9   debt                  21230 non-null  int64  
 10  total_income          21230 non-null  int32  
 11  purpose               21230 non-null  object 
 12  age_group             21230 non-null  object 
 13  median_total_income   21230 non-null  object 
 14  median_days_employed  21230 non-null  float64
dtypes: float64(1), int3

<div class="alert alert-block alert-info">
<b>Senior Reviewer's comment</b> <a class="tocSkip"></a>
    
This column can be deleted.
It isn't a mistake.
</div>

## Categorization of loan purposes

In [93]:
# Observing the distribution of values in loan purposes
data['purpose'].value_counts()

wedding ceremony                            785
having a wedding                            759
to have a wedding                           755
real estate transactions                    669
buy commercial real estate                  655
buying property for renting out             647
transactions with commercial real estate    643
housing transactions                        641
purchase of the house for my family         636
housing                                     635
purchase of the house                       634
property                                    627
construction of own property                626
transactions with my real estate            623
building a property                         619
purchase of my own house                    618
building a real estate                      617
buy real estate                             612
housing renovation                          602
buy residential real estate                 599
buying my own car                       

To optimize the work, we take only unique values of the loan purposes

In [94]:
unique_purpose = data['purpose'].unique()

In [95]:
unique_purpose

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

<div class="alert alert-block alert-info">
<b>Senior Reviewer's comment</b> <a class="tocSkip"></a>
    
Alternative method: 
</div>

The following groups can be formed:

- car transactions (the key word is car)
- real estate transactions (key words: housing, real estate)
- wedding ceremony (keyword: wedding)
- getting an education (keyword: education)



<div class="alert alert-block alert-info">
<b>Senior Reviewer's comment</b> <a class="tocSkip"></a>
    
Standart method: 
</div>

In [138]:
def categorize_purpose(row):
    if 'car' in row['purpose']:
        return 'car'
    if 'hous' in row['purpose'] or 'prop' in row['purpose'] or 'real est' in row['purpose']:
        return 'real estate'
    if 'wedd' in row['purpose']:
        return 'wedding'
    if 'educ' in row['purpose'] or 'uni' in row['purpose']:
        return 'education'


In [139]:
# Creating a column to store the purpose categories
data['purpose_category'] = data.apply(categorize_purpose,axis=1)

In [140]:
# Checking the values of the new column
data['purpose_category'].value_counts()

real estate    10703
car             4258
education       3970
wedding         2299
Name: purpose_category, dtype: int64

## Income level categorization to test the hypothesis

To check whether the loan repayment depends on the client's income level, you need to create categories of the client's income level.
Categorization makes it easier to compare different client groups.


In [142]:
# Getting the data about the column with client income data
data['total_income'].describe().astype('int')

count     21230
mean      26513
std       16094
min        3306
25%       17131
50%       22934
75%       31720
max      499163
Name: total_income, dtype: int32

In [145]:
data[data['total_income'] < 50000]['total_income'].count()

19915

In [146]:
data[(data['total_income'] > 50000) & (data['total_income'] <= 100000)]['total_income'].count()

1215

In [147]:
data[(data['total_income'] > 100000) & (data['total_income'] <= 150000)]['total_income'].count()

71

In [148]:
data[(data['total_income'] > 150000) & (data['total_income'] <= 200000)]['total_income'].count()

17

In [149]:
data[(data['total_income'] > 200000) & (data['total_income'] <= 250000)]['total_income'].count()

5

In [150]:
data[(data['total_income'] > 250000) & (data['total_income'] <= 500000)]['total_income'].count()

7

In [151]:
data[data['total_income'] > 500000]['total_income'].count()

0

In [154]:
# Function to create income levels
def income_level(income):
    if (income > 50000) and (income <= 100000):
        return 'small'
    if (income > 100000) and (income <= 150000):
        return 'avarage'
    if (income > 200000) and (income <= 250000):
        return 'above avarege'
    if (income > 250000) and (income <= 500000):
        return 'high'
    if income > 500000:
        return 'very high'
    
#Also we reccomend to use Letters

In [155]:
# Creating a column to store categories
data['income_level'] = data['total_income'].apply(income_level)

In [156]:
data['income_level'].value_counts()

small            1215
avarage            71
high                7
above avarege       5
Name: income_level, dtype: int64

## Testing the hypotheses

**Is there a connection between having kids and repaying a loan on time?**

In [157]:
# Creating a pivot table
pivot_table_children = data.pivot_table(index='children', columns= 'debt', values='days_employed', aggfunc='count')

In [159]:
# Calculating the percentage of clients that did not return the loan depending on the number of children they have
pivot_table_children['percent_1'] = pivot_table_children[1] / (pivot_table_children[1] + pivot_table_children[0]) * 100

In [160]:
pivot_table_children.sort_values(by='percent_1', ascending=True)

debt,0,1,percent_1
children,,,
0,12963.0,1058.0,7.545824
3,301.0,27.0,8.231707
1,4351.0,441.0,9.202838
2,1845.0,194.0,9.514468
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


**Conclusion**

The percentage of not returned loans for loaners without kids is 1.5% lower than for clients with kid. 
In general, the number of kids itself doesn't affect the probability of repaying the loan.


**Is there a connection between marital status and repaying a loan on time? **

In [161]:
# Creating a pivot table
pivot_table_family_status = data.pivot_table(index='family_status', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_family_status['percent_1'] = pivot_table_family_status[1]/(pivot_table_family_status[1] \
                                                                       + pivot_table_family_status[0]) * 100



In [162]:
# Calculating the percentage of clients that did not return the loan depending on the number of children they have
pivot_table_family_status.sort_values(by='percent_1', ascending=True)

debt,0,1,percent_1
family_status,,,
widow / widower,884,62,6.553911
divorced,1095,84,7.124682
married,11290,923,7.557521
civil partnership,3729,383,9.314202
unmarried,2508,272,9.784173


**Conclusion**

Clients who are not and have never been officially married may more often not repay loans (the share of such debtors is around 2.5% higher than that of other clients).
Unmarried clients are 38% of the total number of debtors.


**Is there a connection between income level and repaying a loan on time?**

In [163]:
# Creating a pivot table
pivot_table_income_level = data.pivot_table(index='income_level', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_income_level['percent_1'] = pivot_table_income_level[1]/(pivot_table_income_level[1] \
                                                                       + pivot_table_income_level[0]) * 100



In [164]:
# Calculating the percentage of clients that did not return the loan depending on the number of children they have
pivot_table_income_level.sort_values(by='percent_1', ascending=True)

debt,0,1,percent_1
income_level,,,
avarage,67.0,4.0,5.633803
small,1129.0,86.0,7.078189
high,6.0,1.0,14.285714
above avarege,5.0,NaN,NaN


**Conclusion**

Clients with monthly income less than 150 thousand, are more likely to not repay loans.
Such clients are almost 72% of the total number of clients who did not timely repay the loan.


**How do different loan purposes affect timely loan repayment?**

In [165]:
# Creating a pivot table
pivot_table_purpose_category = data.pivot_table(index='purpose_category', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_purpose_category['percent_1'] = pivot_table_purpose_category[1]/(pivot_table_purpose_category[1] \
                                                                       + pivot_table_purpose_category[0]) * 100



In [166]:
# Calculating the percentage of clients that did not return the loan depending on the number of children they have
pivot_table_purpose_category.sort_values(by='percent_1', ascending=True)

debt,0,1,percent_1
purpose_category,,,
real estate,9926,777,7.259647
wedding,2118,181,7.872988
education,3601,369,9.294710
car,3861,397,9.323626


**Conclusion**

The highest percentage of not repaid loans is by clients who applied for a loan for education and car transactions.
These clients are 44.4% of the total number of debtors.


# General Conclusions

1. Symmetrical missing values in days_employed and total_income are completely random. 
We need to notify the colleagues responsible for data gathering about this problem. 
Nevertheless, some of these missing values can be explained by certain bank procedures (the loan can be given even without the information on income, only based on the information about the client's property). 
That is why we need clarifications from the colleagues that retrieved this data.

2. Duplicates may have appeared due to a technical issue.
It can also be explained by the fact that there is no standardized form for collecting clients' info. 
Maybe some of the rows weren't duplicates in the dataset - for example, informatio about several clients could be repeated.
That is why we got to us our clients to retrieve the data from the client's ID.
Besides, we have to notify our colleagues about duplicates in the data.

3. One standardized form of writing down the data or some "stop values" introduced by developers, will decrease the amount of time spent on prerpocessing. 
For example, we wouldn't have to deal with negative values in children column.

4. For a new credit scoring system we need to account for the following groups: 
- Clients without children have a lower percentage of repayment delay, than those with children. 
- The most risk-bearing clients are those with monthly income lower than 150 thousand rubles.
- Clients that are not officially married are more likely to not repay the loan in time 
- Moreover, another risk-bearing group for the bank is clients that took a loan for car or education 
